In [1]:
from ship_routing.core import Route, WayPoint
from ship_routing.data import (
    load_currents,
    load_winds,
    load_waves,
)
from ship_routing.algorithms import (
    gradient_descent_across_track_left,
    gradient_descent_along_track,
    gradient_descent_time_shift,
    LargeIncrementError,
    InvalidGradientError,
    ZeroGradientsError,
)

<frozen abc>:106: FutureWarning: xarray subclass HashableDataset should explicitly define __slots__


In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from copy import deepcopy
import tqdm

In [3]:
np.random.seed(345)

In [4]:
from ship_routing.convenience import create_route

In [5]:
route_0 = create_route(
    lon_waypoints=[-10.0, -80.5],
    lat_waypoints=[50.0, 30.0],
    time_start=np.datetime64("2021-08-09"),
    time_end=np.datetime64("2021-08-19"),
    time_resolution_hours=12
)

# route_0 = Route(
#     way_points=(
#         WayPoint(lon=-10, lat=50, time=np.datetime64("2021-08-09")),
#         WayPoint(lon=-80.5, lat=30, time=np.datetime64("2021-08-19")),
#     )
# )
# # refine to 12 hourly way points
# refine_to = np.mean([l.speed_ms for l in route_0.legs]) * 12 * 3600.0
# route_0 = route_0.refine(distance_meters=refine_to)

route_0

Route(way_points=(WayPoint(lon=np.float64(-10.0), lat=np.float64(50.0), time=np.datetime64('2021-08-09')), WayPoint(lon=-14.292201577420826, lat=50.261276172512616, time=np.datetime64('2021-08-09T11:59:59.999')), WayPoint(lon=-18.61708385116704, lat=50.36264428060363, time=np.datetime64('2021-08-09T23:59:59.999')), WayPoint(lon=-22.94574283062347, lat=50.302858182294756, time=np.datetime64('2021-08-10T11:59:59.999')), WayPoint(lon=-27.249088698314868, lat=50.082654353219056, time=np.datetime64('2021-08-10T23:59:59.999')), WayPoint(lon=-31.499266319154454, lat=49.70470817650217, time=np.datetime64('2021-08-11T11:59:59.999')), WayPoint(lon=-35.6709318555244, lat=49.173478785446754, time=np.datetime64('2021-08-11T23:59:59.999')), WayPoint(lon=-39.74225007322045, lat=48.49496299292714, time=np.datetime64('2021-08-12T11:59:59.999')), WayPoint(lon=-43.695532496104526, lat=47.676388700546255, time=np.datetime64('2021-08-12T23:59:59.999')), WayPoint(lon=-47.517500408033804, lat=46.725881673052

In [6]:
from ship_routing.convenience import Logs, LogsRoute

In [7]:
logs_0 = Logs(iteration=0, cost=np.nan, method="initial")
logs_route_0 = LogsRoute(logs=logs_0, route=route_0)
logs_route_0.data_frame

,iteration,cost,method,lon,lat,time
0,0,NaN,initial,-10.000000,50.000000,2021-08-09 00:00:00.000
1,0,NaN,initial,-14.292202,50.261276,2021-08-09 11:59:59.999
2,0,NaN,initial,-18.617084,50.362644,2021-08-09 23:59:59.999
3,0,NaN,initial,-22.945743,50.302858,2021-08-10 11:59:59.999
4,0,NaN,initial,-27.249089,50.082654,2021-08-10 23:59:59.999
5,0,NaN,initial,-31.499266,49.704708,2021-08-11 11:59:59.999
6,0,NaN,initial,-35.670932,49.173479,2021-08-11 23:59:59.999
7,0,NaN,initial,-39.742250,48.494963,2021-08-12 11:59:59.999
8,0,NaN,initial,-43.695532,47.676389,2021-08-12 23:59:59.999
9,0,NaN,initial,-47.517500,46.725882,2021-08-13 11:59:59.999


In [9]:
%pip install jsonpickle

Note: you may need to restart the kernel to use updated packages.


In [10]:
import jsonpickle

In [15]:
jsonpickle.dumps?

Signature:
jsonpickle.dumps(
    value,
    unpicklable=True,
    make_refs=True,
    keys=False,
    max_depth=None,
    reset=True,
    backend=None,
    warn=False,
    context=None,
    max_iter=None,
    use_decimal=False,
    numeric_keys=False,
    use_base85=False,
    fail_safe=None,
    indent=None,
    separators=None,
    include_properties=False,
    handle_readonly=False,
)
Docstring:
Return a JSON formatted representation of value, a Python object.

:param unpicklable: If set to ``False`` then the output will not contain the
    information necessary to turn the JSON data back into Python objects,
    but a simpler JSON stream is produced. It's recommended to set this
    parameter to ``False`` when your code does not rely on two objects
    having the same ``id()`` value, and when it is sufficient for those two
    objects to be equal by ``==``, such as when serializing sklearn
    instances. If you experience (de)serialization being incorrect when you
    use numpy, pa

In [13]:
jsonpickle.loads(jsonpickle.dumps([logs_route_0 for n in range(12)]))

[LogsRoute(logs=Logs(iteration=0, cost=nan, method='initial'), route=Route(way_points=(WayPoint(lon=np.float64(-10.0), lat=np.float64(50.0), time=np.datetime64('2021-08-09')), WayPoint(lon=-14.292201577420826, lat=50.261276172512616, time=np.datetime64('2021-08-09T11:59:59.999')), WayPoint(lon=-18.61708385116704, lat=50.36264428060363, time=np.datetime64('2021-08-09T23:59:59.999')), WayPoint(lon=-22.94574283062347, lat=50.302858182294756, time=np.datetime64('2021-08-10T11:59:59.999')), WayPoint(lon=-27.249088698314868, lat=50.082654353219056, time=np.datetime64('2021-08-10T23:59:59.999')), WayPoint(lon=-31.499266319154454, lat=49.70470817650217, time=np.datetime64('2021-08-11T11:59:59.999')), WayPoint(lon=-35.6709318555244, lat=49.173478785446754, time=np.datetime64('2021-08-11T23:59:59.999')), WayPoint(lon=-39.74225007322045, lat=48.49496299292714, time=np.datetime64('2021-08-12T11:59:59.999')), WayPoint(lon=-43.695532496104526, lat=47.676388700546255, time=np.datetime64('2021-08-12T2